- 이 노트북 파일(ffc.ipynb)는 파이썬과 스타타를 같이 사용하는 예를 보이기 위한 것임
- 노트북 파일 실행과정에서 불러오는 피클, 함수 등이 없으므로, 실행시 오류 발생

## 변수 생성

## [STATA .dta 파일 export](#export)

## [STATA 분석 결과 import](#stata_result)

## 노동생산성 계산(VA0 이용)

- non-null count: VA 303833, VA0 454195, VA1 429453, VA2 428807

In [105]:
kis = pd.read_pickle('pickle//kis_trim.pkl')

## 자산5분위(같은해, 같은 중분류내): qasset

- 자산총계 == nan 인 기업 제거(2017년말 기준 126개 기업)
- 자산분위 == nan 인 기업 제거(전체 기간중 14개 기업, 산업분류없음)
    - KSIC2 == 36인 기업 1개, KSIC2 값이 비어있는 기업 1개 제거

In [106]:
kis = kis.loc[~kis.자산총계.isnull() & ~kis.KSIC2.isnull(), :]

q = '자산총계'
kis.loc[~kis[q].isnull(), 'qasset'] = kis[~kis[q].isnull()].groupby(['DATE', 'KSIC2'])[q].apply(quintile)
kis.loc[:, 'qasset'] += 1  # 분위값이 1~5의 값을 갖도록 1을 더함

kis = kis.loc[~kis.qasset.isnull()]

## 노동생산성 5분위(같은해, 같은 중분류내): qvl

- 노동생산성 상/하위 60, 80% 변수생성(ple3, ple4, pge2, pge3)

In [107]:
kis.loc[kis.VA0.le(0), 'VA0'] = np.nan
kis['VL'] = kis['VA0'] / kis['종업원수']

q = 'VL'
kis.loc[~kis[q].isnull(), 'qvl'] = kis[~kis[q].isnull()].groupby(['DATE', 'KSIC2'])[q].apply(quintile)
kis.loc[:, 'qvl'] += 1  # 분위값이 1~5의 값을 갖도록 1을 더함

In [108]:
kis.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 495170 entries, 0 to 496315
Columns: 109 entries, KIS to qvl
dtypes: datetime64[ns](3), float64(93), int32(1), int64(1), object(11)
memory usage: 413.7+ MB


In [109]:
kis = kis[['KIS', 'DATE', 'AGE', 'KSIC', 'KSIC2', '주거래은행', '시장', '기업규모', 'DEBT', 
           '자산총계', '자본총계', '매출액', '유형자산', '유동자산', '비유동자산', 'EBITDA', '현금', '감가상각비',
           'qasset', 'VL', 'qvl', 'ICR']]

# 최근 3년간 노동생산성 분위값, 이자보상비율에 따라 기업 분류

- 1, 2년전 노동생산성, 이자보상비율 변수 생성

In [110]:
shifts = kis[['KIS', 'DATE', 'qvl', 'ICR']].set_index(['KIS', 'DATE'])

In [111]:
shifts = shifts.unstack(level = 0)

In [112]:
for l in [1, 2, 3]:
    df = shifts.shift(l)
    df = df.stack()
    df.columns = [col + str(l) for col in df.columns]
    df = df.reset_index()
    kis = kis.merge(df, on = ['KIS', 'DATE'], how = 'left')

## 지난 3년간 노동생산성 더미 변수

- prod0: 모든 기업
- prod1: VL is not null
- prod2: 지난 3년간 노동생산성 하위 60%
- prod3: 지난 3년간 노동생산성 하위 80%
- prod4: 지난 3년간 노동생산성 상위 80%
- prod5: 지난 3년간 노동생산성 상위 60%
- prod6: not prod2
- prod7: not prod5

In [113]:
kis['prod0'] = 0
kis.loc[:, 'prod0'] = 1
kis['prod1'] = 0
kis.loc[~kis.VL.isnull(), 'prod1'] = 1
kis['prod2'] = 0
kis.loc[kis.qvl.le(3) & kis.qvl1.le(3) & kis.qvl2.le(3) & ~kis.VL.isnull(), 'prod2'] = 1
kis['prod3'] = 0
kis.loc[kis.qvl.le(4) & kis.qvl1.le(4) & kis.qvl2.le(4) & ~kis.VL.isnull(), 'prod3'] = 1
kis['prod4'] = 0
kis.loc[kis.qvl.ge(2) & kis.qvl1.ge(2) & kis.qvl2.ge(2) & ~kis.VL.isnull(), 'prod4'] = 1
kis['prod5'] = 0
kis.loc[kis.qvl.ge(3) & kis.qvl1.ge(3) & kis.qvl2.ge(3) & ~kis.VL.isnull(), 'prod5'] = 1
kis['prod6'] = 0
kis.loc[kis.prod2.ne(1) &  ~kis.VL.isnull(), 'prod6'] = 1
kis['prod7'] = 0
kis.loc[kis.prod5.ne(1) &  ~kis.VL.isnull(), 'prod7'] = 1

### 지난 3년 내내 icr < 1: ICR3 == 3

- replace mf0 = 1              // 모든 기업
- replace mf1 = 1 if ICR3 == 0 // 한계기업 아님: 3년(Y~Y-2)동안 0번 ICR < 1    (59.1%)
- replace mf2 = 1 if ICR3 <= 1 // 한계기업 아님: 3년(Y~Y-2)동안 1번이하 ICR < 1 (74.3%)
- replace mf3 = 1 if ICR3 >= 2 // 준한계기업: 3년(Y~Y-2)동안 2번이상 ICR < 1    (14.5%)
- replace mf4 = 1 if ICR3 == 3 // 한계기업: 3년(Y~Y-2)동안 3번 ICR < 1         (6.4%)

In [114]:
kis['mf0'] = 1
kis['mf1'] = 0
kis.loc[kis.ICR.ge(1) & kis.ICR1.ge(1) & kis.ICR2.ge(1), 'mf1'] = 1
kis['mf2'] = 0
kis.loc[kis.ICR.lt(1).astype(int) + kis.ICR1.lt(1).astype(int) + kis.ICR2.lt(1).astype(int) <= 1, 'mf2'] = 1
kis['mf3'] = 0
kis.loc[kis.ICR.lt(1).astype(int) + kis.ICR1.lt(1).astype(int) + kis.ICR2.lt(1).astype(int) <= 2, 'mf3'] = 1
kis['mf4'] = 0
kis.loc[kis.ICR.lt(1) & kis.ICR1.lt(1) & kis.ICR2.lt(1), 'mf4'] = 1

In [115]:
kis['qvl0'] = 0
kis.loc[:, 'qvl0'] = 1
kis['qvl1'] = 0
kis.loc[kis.qvl.ge(1) & ~kis.qvl.isnull(), 'qvl1'] = 1
kis['qvl2'] = 0
kis.loc[kis.qvl.ge(2) & ~kis.qvl.isnull(), 'qvl2'] = 1
kis['qvl3'] = 0
kis.loc[kis.qvl.ge(3) & ~kis.qvl.isnull(), 'qvl3'] = 1
kis['qvl4'] = 0
kis.loc[kis.qvl.le(4) & ~kis.qvl.isnull(), 'qvl4'] = 1
kis['qvl5'] = 0
kis.loc[kis.qvl.le(3) & ~kis.qvl.isnull(), 'qvl5'] = 1

## SA, EA, liq

In [116]:
kis['sa'] = kis['매출액'] / kis['자산총계']     # 토빈q, 자본 한계생산성 대리변수(정대희 2013)
kis['liq'] = kis['유동자산'] / kis['자산총계']   # 유동자산비율 (Duchin et al 2010)
kis['ea'] = kis['EBITDA'] / kis['자산총계']    # 현금흐름(기업투자 관련 문헌에서 자주 사용되는..., 정대희 2013)
kis['cash'] = kis['현금'] / kis['자산총계']    # 

# STATA .dta 파일 export

In [124]:
kis = kis[kis.자본총계 > 0]      # 2014년 기준 26808개 기업중 3056개 기업의 자본총계가 마이너스
kis = kis[kis.매출액 > 0]        # 2014년 기준 10건이  < 0 
kis = kis[kis.유동자산 > 0]        # 2014년 기준 4건이 < 0 
kis = kis[kis.비유동자산 > 0]        # 2014년 기준 1건이 < 0 

age_qvl = ['AGE', 'sa', 'ea', 'cash', 'liq', 'qasset', 'VL', 'qvl']
mf_prod = (['mf' + str(i) for i in range(0, 5)] + ['prod' + str(i) for i in range(0, 8)] 
           + ['qvl' + str(i) for i in range(0, 6)])

dta = kis[['DATE', 'KIS', 'KSIC', 'KSIC2', '주거래은행', '시장', '기업규모', 'DEBT', '자산총계', '감가상각비', 
           '매출액', '유형자산'] + age_qvl + mf_prod].copy()

dta.columns = ['DATE', 'KIS', 'KSIC', 'KSIC2', 'BANK', 'MKT', 'SIZE', 'debt', 'asset', 'dep', 
               'sales', 'TASSET'] + age_qvl + mf_prod

dta = dta.drop(['BANK', 'MKT'], axis = 1)

# # 소속시장, 현재 기업 상태를 영문으로 변경
# mkt_k = ['코스닥상장', '외감', '코스닥관리', '상장', '코넥스', '상장관리', '폐업', '피흡수합병', '일반',
#          '공기관', '기타', '개인', '비영리단체']
# mkt_e = ['KQ', 'EA', 'KQ1', 'KI', 'KX', 'KI1', 'CL', 'MA', 'OD', 'PU', 'OT', 'PR', 'NP']
# dta.loc[:, 'MKT'] = dta.loc[:, 'MKT'].replace(dict(zip(mkt_k, mkt_e)))

# # 14년 취약성 수준(BIS 비율)에 따라 국민-신한-하나-우리-산업-기업:1-2-3-4-5-6 으로 지정
# dta['BANK'] = dta['BANK'].replace(dict(zip(['국민', '신한', '하나', '우리', '산업', '기업'], [0, 1, 2, 3, 4, 5])))
# dta.loc[~dta.BANK.isin([0, 1, 2, 3, 4, 5]), 'BANK'] = 6

dta.loc[:, ['SIZE']] = dta.loc[:, ['SIZE']].replace({'대기업':0, '중소기업':1, '기타':2})

dta.DATE = pd.DatetimeIndex(dta.DATE).year.astype(str)
dta.DATE = dta.DATE.str[2:]

dta = dta.set_index(['DATE', 'KIS', 'KSIC', 'KSIC2', 'SIZE'])

dta.to_stata('stata//ffc.dta')

<a id='stata_result'></a>

# STATA 분석 결과 불러오기

# [STATA 분석 결과 전체 제조업]

# [보고서 표4](#T4)

<a id='T4'></a>

# 보고서 <표 4>

- 자산규모 하위 80%만 17년도 유의
- 저업력은 15년, 고업력은 17년 유의

In [2]:
fs = [0, 9, 10, 11, 12]

In [3]:
mod = 'ms_0_0_0_ea_4'

df_stata = get_duchin(mod, sec = [1], smpl = fs, ys = range(8, 18))

be = df_stata.unstack().stack(level = 0)
pv = be.loc[idx[['X', 'id'], ['p']], idx[1, fs, range(8, 18)]]
be = be.loc[idx[['X', 'id'], ['b', 'e']], idx[1, fs, range(8, 18)]]
be = be.rename(index=order_dict, level = 0).sort_index(axis = 0)
pv = pv.rename(index=order_dict, level = 0).sort_index(axis = 0)
be = be.astype(float)

be = be.stack(level = 1)
be = be.reorder_levels([0, 2, 1], axis = 0).sort_index(axis = 0)

pv = pv.stack(level = 1)
pv = pv.reorder_levels([0, 2, 1], axis = 0).sort_index(axis = 0)

be.to_excel('output//' + mod + '_asset_age.xlsx')

display((df_stata['b'].loc['X'].applymap(dp4) * (df_stata['p'].loc['X'] >= 1)).T)
display_side_by_side(pv.loc['1X'])

year          8       9       10 11 12 13      14 15      16 17
sec smpl                                                       
1   0     0.0947  0.0731  0.0485                      0.0416   
    9     0.1250  0.0521  0.0524                      0.0486   
    10            0.1250                                       
    11    0.1060                           0.1250              
    12    0.0835  0.0999  0.0481                      0.0519

sec 
 1 
 
 
 
 year 
 8 
 9 
 10 
 11 
 12 
 13 
 14 
 15 
 16 
 17 
 
 
 smpl 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 p 
 3 
 3 
 2 
 0 
 0 
 0 
 0 
 0 
 3 
 0 
 
 
 9 
 p 
 3 
 2 
 2 
 0 
 0 
 0 
 0 
 0 
 2 
 0 
 
 
 10 
 p 
 0 
 3 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 11 
 p 
 3 
 0 
 0 
 0 
 0 
 0 
 3 
 0 
 0 
 0 
 
 
 12 
 p 
 3 
 3 
 2 
 0 
 0 
 0 
 0 
 0 
 3 
 0

In [4]:
df_stata

b                                e                             \
sec          1                                1                              
smpl        0     9     10    11    12       0        9        10       11   
var year                                                                     
D   8    -0.02 -0.02 -0.02 -0.01 -0.02  0.00398  0.00469  0.00653  0.00666   
    9     0.01  0.02  0.00  0.02  0.01  0.00247  0.00306  0.00397  0.00564   
    10    0.01  0.01  0.01  0.01  0.01  0.00251  0.00313  0.00404  0.00596   
    11    0.01  0.01  0.00  0.01  0.01  0.00214  0.00271  0.00333  0.00587   
    12    0.00  0.01  0.00  0.01  0.00  0.00189  0.00245  0.00273  0.00513   
...        ...   ...   ...   ...   ...      ...      ...      ...      ...   
sa  13    0.01  0.02 -0.00  0.02  0.01  0.00542  0.00632  0.00984   0.0105   
    14    0.03  0.03  0.01  0.04  0.02  0.00696  0.00796   0.0138   0.0142   
    15    0.04  0.04  0.04  0.04  0.04  0.00582  0.00680   0.0100   0.0109   
    16    0.04  0.04  0.02  0.04  0.04  0.00613  0.00709   0.0105   0.0132   
    17    0.03  0.03  0.00  0.04  0.03  0.00504  0.00565  0.00967   0.0108   

                   p              
sec                1              
smpl           12 0  9  10 11 12  
var year                          
D   8     0.00468  3  3  3  1  3  
    9     0.00269  3  3  0  3  3  
    10    0.00274  3  3  3  2  3  
    11    0.00226  3  3  0  2  3  
    12    0.00203  2  3  0  2  0  
...           ... .. .. .. .. ..  
sa  13    0.00627  2  3  0  0  1  
    14    0.00795  3  3  0  3  3  
    15    0.00673  3  3  3  3  3  
    16    0.00674  3  3  2  3  3  
    17    0.00560  3  3  0  3  3  

[90 rows x 15 columns]

In [3]:
# df_stata['p']

## <부표 1a>

In [13]:
mod = 'ms_0_6_0_ea_4'

df_stata = get_duchin(mod, sec = [1], smpl = fs, ys = range(8, 18))

be = df_stata.unstack().stack(level = 0)
pv = be.loc[idx[['X', 'id'], ['p']], idx[1, fs, range(8, 18)]]
be = be.loc[idx[['X', 'id'], ['b', 'e']], idx[1, fs, range(8, 18)]]
be = be.rename(index=order_dict, level = 0).sort_index(axis = 0)
pv = pv.rename(index=order_dict, level = 0).sort_index(axis = 0)
be = be.astype(float)

be = be.stack(level = 1)
be = be.reorder_levels([0, 2, 1], axis = 0).sort_index(axis = 0)

pv = pv.stack(level = 1)
pv = pv.reorder_levels([0, 2, 1], axis = 0).sort_index(axis = 0)

be.to_excel('output//' + mod + '_asset_age.xlsx')

display((df_stata['b'].loc['X'].applymap(dp4) * (df_stata['p'].loc['X'] >= 1)).T)
display_side_by_side(pv.loc['1X'])

year          8       9       10 11 12 13      14       15      16 17
sec smpl                                                             
1   0     0.0714  0.0545                                    0.0567   
    9     0.1250          0.0626                            0.0715   
    10            0.0708                                             
    11    0.0831                           0.1540  -0.0854           
    12    0.0612  0.0680                                    0.0608

sec 
 1 
 
 
 
 year 
 8 
 9 
 10 
 11 
 12 
 13 
 14 
 15 
 16 
 17 
 
 
 smpl 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 p 
 2 
 2 
 0 
 0 
 0 
 0 
 0 
 0 
 3 
 0 
 
 
 9 
 p 
 3 
 0 
 1 
 0 
 0 
 0 
 0 
 0 
 3 
 0 
 
 
 10 
 p 
 0 
 2 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 11 
 p 
 1 
 0 
 0 
 0 
 0 
 0 
 3 
 1 
 0 
 0 
 
 
 12 
 p 
 1 
 2 
 0 
 0 
 0 
 0 
 0 
 0 
 3 
 0

In [14]:
be

sec               1                                                        \
year             8        9        10       11       12       13       14   
var smpl                                                                    
1X  0    b     0.07     0.05     0.04    -0.00     0.00     0.02     0.03   
         e     0.03     0.02     0.02     0.02     0.02     0.02     0.02   
    9    b     0.12     0.05     0.06    -0.01     0.02     0.03     0.03   
         e     0.04     0.03     0.03     0.03     0.03     0.03     0.03   
    10   b    -0.01     0.07    -0.00     0.01    -0.02    -0.00     0.03   
         e     0.04     0.03     0.03     0.03     0.03     0.03     0.03   
    11   b     0.08     0.03     0.05    -0.03     0.02    -0.03     0.15   
         e     0.05     0.05     0.05     0.05     0.04     0.05     0.05   
    12   b     0.06     0.07     0.03     0.01    -0.00     0.04    -0.01   
         e     0.04     0.03     0.03     0.02     0.02     0.02     0.02   
5id 0    b 4,273.00 4,316.00 4,367.00 4,505.00 4,710.00 4,608.00 5,011.00   
         e     0.00     0.00     0.00     0.00     0.00     0.00     0.00   
    9    b 2,594.00 2,710.00 2,700.00 2,767.00 2,969.00 2,991.00 3,271.00   
         e     0.00     0.00     0.00     0.00     0.00     0.00     0.00   
    10   b 1,679.00 1,606.00 1,667.00 1,738.00 1,741.00 1,617.00 1,740.00   
         e     0.00     0.00     0.00     0.00     0.00     0.00     0.00   
    11   b 1,261.00 1,209.00 1,052.00   922.00   951.00   893.00   988.00   
         e     0.00     0.00     0.00     0.00     0.00     0.00     0.00   
    12   b 3,012.00 3,107.00 3,315.00 3,583.00 3,759.00 3,715.00 4,023.00   
         e     0.00     0.00     0.00     0.00     0.00     0.00     0.00   

sec                                    
year             15       16       17  
var smpl                               
1X  0    b    -0.02     0.06     0.01  
         e     0.02     0.02     0.02  
    9    b    -0.03     0.07     0.02  
         e     0.03     0.03     0.02  
    10   b    -0.01     0.02    -0.03  
         e     0.03     0.03     0.03  
    11   b    -0.09     0.04     0.03  
         e     0.05     0.05     0.04  
    12   b     0.00     0.06     0.00  
         e     0.02     0.02     0.02  
5id 0    b 5,103.00 5,313.00 5,617.00  
         e     0.00     0.00     0.00  
    9    b 3,301.00 3,586.00 3,929.00  
         e     0.00     0.00     0.00  
    10   b 1,802.00 1,727.00 1,688.00  
         e     0.00     0.00     0.00  
    11   b 1,034.00 1,057.00 1,108.00  
         e     0.00     0.00     0.00  
    12   b 4,069.00 4,256.00 4,509.00  
         e     0.00     0.00     0.00

## <부표 1b>

In [18]:
mod = 'ms_0_2_0_ea_4'

df_stata = get_duchin(mod, sec = [1], smpl = fs, ys = range(8, 18))

be = df_stata.unstack().stack(level = 0)
pv = be.loc[idx[['X', 'id'], ['p']], idx[1, fs, range(8, 18)]]
be = be.loc[idx[['X', 'id'], ['b', 'e']], idx[1, fs, range(8, 18)]]
be = be.rename(index=order_dict, level = 0).sort_index(axis = 0)
pv = pv.rename(index=order_dict, level = 0).sort_index(axis = 0)
be = be.astype(float)

be = be.stack(level = 1)
be = be.reorder_levels([0, 2, 1], axis = 0).sort_index(axis = 0)

pv = pv.stack(level = 1)
pv = pv.reorder_levels([0, 2, 1], axis = 0).sort_index(axis = 0)

be.to_excel('output//' + mod + '_asset_age.xlsx')

display((df_stata['b'].loc['X'].applymap(dp4) * (df_stata['p'].loc['X'] >= 1)).T)
display_side_by_side(pv.loc['1X'])

year          8       9       10 11 12      13 14 15 16 17
sec smpl                                                  
1   0     0.1220  0.1190                                  
    9     0.1010                                          
    10    0.1930  0.3600  0.1530                          
    11                                                    
    12    0.1210  0.1950                0.0548

sec 
 1 
 
 
 
 year 
 8 
 9 
 10 
 11 
 12 
 13 
 14 
 15 
 16 
 17 
 
 
 smpl 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 p 
 2 
 3 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 9 
 p 
 1 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 10 
 p 
 1 
 3 
 1 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 11 
 p 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 12 
 p 
 2 
 3 
 0 
 0 
 0 
 1 
 0 
 0 
 0 
 0

In [19]:
be

sec               1                                                        \
year             8        9        10       11       12       13       14   
var smpl                                                                    
1X  0    b     0.12     0.12     0.02    -0.03    -0.00     0.03    -0.03   
         e     0.05     0.04     0.04     0.03     0.03     0.03     0.03   
    9    b     0.10     0.06    -0.00    -0.04    -0.00     0.05    -0.04   
         e     0.06     0.04     0.04     0.04     0.04     0.03     0.04   
    10   b     0.19     0.36     0.15     0.02     0.02    -0.03     0.00   
         e     0.10     0.08     0.09     0.07     0.05     0.05     0.05   
    11   b     0.11    -0.10    -0.05     0.01    -0.11    -0.07    -0.01   
         e     0.09     0.08     0.08     0.09     0.09     0.08     0.08   
    12   b     0.12     0.20     0.05    -0.04     0.03     0.05    -0.03   
         e     0.06     0.04     0.04     0.03     0.03     0.03     0.03   
5id 0    b 2,016.00 2,055.00 2,241.00 2,318.00 2,469.00 2,599.00 2,768.00   
         e     0.00     0.00     0.00     0.00     0.00     0.00     0.00   
    9    b 1,600.00 1,661.00 1,807.00 1,852.00 1,908.00 1,990.00 2,128.00   
         e     0.00     0.00     0.00     0.00     0.00     0.00     0.00   
    10   b   416.00   394.00   434.00   466.00   561.00   609.00   640.00   
         e     0.00     0.00     0.00     0.00     0.00     0.00     0.00   
    11   b   429.00   433.00   424.00   341.00   325.00   318.00   345.00   
         e     0.00     0.00     0.00     0.00     0.00     0.00     0.00   
    12   b 1,587.00 1,622.00 1,817.00 1,977.00 2,144.00 2,281.00 2,423.00   
         e     0.00     0.00     0.00     0.00     0.00     0.00     0.00   

sec                                    
year             15       16       17  
var smpl                               
1X  0    b     0.02     0.00     0.04  
         e     0.03     0.03     0.02  
    9    b    -0.00    -0.00     0.04  
         e     0.04     0.03     0.03  
    10   b     0.08     0.03     0.01  
         e     0.06     0.06     0.05  
    11   b     0.04    -0.10     0.11  
         e     0.09     0.08     0.07  
    12   b     0.02     0.02     0.02  
         e     0.03     0.03     0.03  
5id 0    b 2,809.00 3,029.00 3,096.00  
         e     0.00     0.00     0.00  
    9    b 2,192.00 2,360.00 2,443.00  
         e     0.00     0.00     0.00  
    10   b   617.00   669.00   653.00  
         e     0.00     0.00     0.00  
    11   b   345.00   362.00   346.00  
         e     0.00     0.00     0.00  
    12   b 2,464.00 2,667.00 2,750.00  
         e     0.00     0.00     0.00

In [20]:
mod = 'ms_0_3_0_ea_4'

df_stata = get_duchin(mod, sec = [1], smpl = fs, ys = range(8, 18))

be = df_stata.unstack().stack(level = 0)
pv = be.loc[idx[['X', 'id'], ['p']], idx[1, fs, range(8, 18)]]
be = be.loc[idx[['X', 'id'], ['b', 'e']], idx[1, fs, range(8, 18)]]
be = be.rename(index=order_dict, level = 0).sort_index(axis = 0)
pv = pv.rename(index=order_dict, level = 0).sort_index(axis = 0)
be = be.astype(float)

be = be.stack(level = 1)
be = be.reorder_levels([0, 2, 1], axis = 0).sort_index(axis = 0)

pv = pv.stack(level = 1)
pv = pv.reorder_levels([0, 2, 1], axis = 0).sort_index(axis = 0)

be.to_excel('output//' + mod + '_asset_age.xlsx')

display((df_stata['b'].loc['X'].applymap(dp4) * (df_stata['p'].loc['X'] >= 1)).T)
display_side_by_side(pv.loc['1X'])

year          8       9  10       11 12      13 14 15      16      17
sec smpl                                                             
1   0     0.1210  0.0924                               0.0396  0.0314
    9     0.1100             -0.0466                           0.0420
    10    0.1410  0.2390                                             
    11    0.1060                                                     
    12    0.1270  0.1450                 0.0576        0.0457

sec 
 1 
 
 
 
 year 
 8 
 9 
 10 
 11 
 12 
 13 
 14 
 15 
 16 
 17 
 
 
 smpl 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 p 
 3 
 3 
 0 
 0 
 0 
 0 
 0 
 0 
 1 
 1 
 
 
 9 
 p 
 3 
 0 
 0 
 1 
 0 
 0 
 0 
 0 
 0 
 1 
 
 
 10 
 p 
 2 
 3 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 11 
 p 
 1 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 12 
 p 
 3 
 3 
 0 
 0 
 0 
 2 
 0 
 0 
 2 
 0

In [21]:
be

sec               1                                                        \
year             8        9        10       11       12       13       14   
var smpl                                                                    
1X  0    b     0.12     0.09     0.02    -0.03    -0.00     0.03    -0.01   
         e     0.03     0.03     0.03     0.02     0.02     0.02     0.02   
    9    b     0.11     0.05     0.03    -0.05    -0.00     0.04    -0.00   
         e     0.04     0.03     0.03     0.03     0.03     0.03     0.03   
    10   b     0.14     0.24     0.01     0.03     0.01     0.01    -0.02   
         e     0.06     0.05     0.06     0.05     0.04     0.04     0.04   
    11   b     0.11    -0.04     0.05    -0.02    -0.07    -0.08     0.10   
         e     0.06     0.05     0.06     0.06     0.06     0.06     0.07   
    12   b     0.13     0.14     0.02    -0.03     0.02     0.06    -0.03   
         e     0.04     0.03     0.03     0.03     0.02     0.02     0.03   
5id 0    b 3,594.00 3,598.00 3,862.00 3,994.00 4,211.00 4,283.00 4,514.00   
         e     0.00     0.00     0.00     0.00     0.00     0.00     0.00   
    9    b 2,665.00 2,731.00 2,914.00 2,957.00 3,030.00 3,081.00 3,288.00   
         e     0.00     0.00     0.00     0.00     0.00     0.00     0.00   
    10   b   929.00   867.00   948.00 1,037.00 1,181.00 1,202.00 1,226.00   
         e     0.00     0.00     0.00     0.00     0.00     0.00     0.00   
    11   b   837.00   807.00   765.00   631.00   603.00   559.00   595.00   
         e     0.00     0.00     0.00     0.00     0.00     0.00     0.00   
    12   b 2,757.00 2,791.00 3,097.00 3,363.00 3,608.00 3,724.00 3,919.00   
         e     0.00     0.00     0.00     0.00     0.00     0.00     0.00   

sec                                    
year             15       16       17  
var smpl                               
1X  0    b     0.01     0.04     0.03  
         e     0.02     0.02     0.02  
    9    b    -0.01     0.03     0.04  
         e     0.03     0.02     0.02  
    10   b     0.05     0.06    -0.01  
         e     0.04     0.04     0.03  
    11   b    -0.02     0.01     0.08  
         e     0.07     0.06     0.05  
    12   b     0.02     0.05     0.02  
         e     0.02     0.02     0.02  
5id 0    b 4,674.00 4,954.00 5,044.00  
         e     0.00     0.00     0.00  
    9    b 3,435.00 3,662.00 3,801.00  
         e     0.00     0.00     0.00  
    10   b 1,239.00 1,292.00 1,243.00  
         e     0.00     0.00     0.00  
    11   b   619.00   618.00   599.00  
         e     0.00     0.00     0.00  
    12   b 4,055.00 4,336.00 4,445.00  
         e     0.00     0.00     0.00

In [16]:
mod = 'ms_0_7_0_ea_4'

df_stata = get_duchin(mod, sec = [1], smpl = fs, ys = range(8, 18))

be = df_stata.unstack().stack(level = 0)
pv = be.loc[idx[['X', 'id'], ['p']], idx[1, fs, range(8, 18)]]
be = be.loc[idx[['X', 'id'], ['b', 'e']], idx[1, fs, range(8, 18)]]
be = be.rename(index=order_dict, level = 0).sort_index(axis = 0)
pv = pv.rename(index=order_dict, level = 0).sort_index(axis = 0)
be = be.astype(float)

be = be.stack(level = 1)
be = be.reorder_levels([0, 2, 1], axis = 0).sort_index(axis = 0)

pv = pv.stack(level = 1)
pv = pv.reorder_levels([0, 2, 1], axis = 0).sort_index(axis = 0)

be.to_excel('output//' + mod + '_asset_age.xlsx')

display((df_stata['b'].loc['X'].applymap(dp4) * (df_stata['p'].loc['X'] >= 1)).T)
display_side_by_side(pv.loc['1X'])

year          8       9  10 11 12       13 14 15      16 17
sec smpl                                                   
1   0     0.1080  0.0684                                   
    9     0.1230                                           
    10            0.2120                                   
    11    0.1050                   -0.1050                 
    12    0.1010  0.1200            0.0430        0.0438

sec 
 1 
 
 
 
 year 
 8 
 9 
 10 
 11 
 12 
 13 
 14 
 15 
 16 
 17 
 
 
 smpl 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 p 
 3 
 2 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 9 
 p 
 3 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 10 
 p 
 0 
 3 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 11 
 p 
 1 
 0 
 0 
 0 
 0 
 2 
 0 
 0 
 0 
 0 
 
 
 12 
 p 
 2 
 3 
 0 
 0 
 0 
 1 
 0 
 0 
 1 
 0

In [17]:
be

sec               1                                                        \
year             8        9        10       11       12       13       14   
var smpl                                                                    
1X  0    b     0.11     0.07     0.04    -0.02     0.02     0.01     0.01   
         e     0.03     0.03     0.03     0.02     0.02     0.02     0.02   
    9    b     0.12     0.02     0.04    -0.02     0.01     0.01     0.01   
         e     0.04     0.03     0.03     0.03     0.03     0.03     0.03   
    10   b     0.05     0.21     0.05    -0.05     0.03    -0.02     0.00   
         e     0.06     0.05     0.06     0.05     0.04     0.04     0.04   
    11   b     0.10    -0.04     0.04    -0.01     0.00    -0.10     0.08   
         e     0.06     0.05     0.06     0.06     0.05     0.05     0.06   
    12   b     0.10     0.12     0.04    -0.03     0.02     0.04    -0.01   
         e     0.04     0.03     0.03     0.03     0.02     0.02     0.03   
5id 0    b 3,555.00 3,587.00 3,711.00 3,926.00 4,219.00 4,261.00 4,702.00   
         e     0.00     0.00     0.00     0.00     0.00     0.00     0.00   
    9    b 2,787.00 2,830.00 2,905.00 3,035.00 3,273.00 3,327.00 3,605.00   
         e     0.00     0.00     0.00     0.00     0.00     0.00     0.00   
    10   b   768.00   757.00   806.00   891.00   946.00   934.00 1,097.00   
         e     0.00     0.00     0.00     0.00     0.00     0.00     0.00   
    11   b   954.00   925.00   839.00   713.00   747.00   729.00   850.00   
         e     0.00     0.00     0.00     0.00     0.00     0.00     0.00   
    12   b 2,601.00 2,662.00 2,872.00 3,213.00 3,472.00 3,532.00 3,852.00   
         e     0.00     0.00     0.00     0.00     0.00     0.00     0.00   

sec                                    
year             15       16       17  
var smpl                               
1X  0    b    -0.01     0.03     0.01  
         e     0.02     0.02     0.02  
    9    b    -0.01     0.03     0.02  
         e     0.03     0.03     0.02  
    10   b    -0.02     0.05    -0.03  
         e     0.04     0.04     0.04  
    11   b    -0.03    -0.01     0.06  
         e     0.06     0.06     0.04  
    12   b    -0.00     0.04    -0.00  
         e     0.02     0.02     0.02  
5id 0    b 4,790.00 5,027.00 5,374.00  
         e     0.00     0.00     0.00  
    9    b 3,623.00 3,958.00 4,357.00  
         e     0.00     0.00     0.00  
    10   b 1,167.00 1,069.00 1,017.00  
         e     0.00     0.00     0.00  
    11   b   872.00   899.00   937.00  
         e     0.00     0.00     0.00  
    12   b 3,918.00 4,128.00 4,437.00  
         e     0.00     0.00     0.00

## <부표 2a>

In [22]:
mod = 'ms_2_0_0_ea_4'

df_stata = get_duchin(mod, sec = [1], smpl = fs, ys = range(8, 18))

be = df_stata.unstack().stack(level = 0)
pv = be.loc[idx[['X', 'id'], ['p']], idx[1, fs, range(8, 18)]]
be = be.loc[idx[['X', 'id'], ['b', 'e']], idx[1, fs, range(8, 18)]]
be = be.rename(index=order_dict, level = 0).sort_index(axis = 0)
pv = pv.rename(index=order_dict, level = 0).sort_index(axis = 0)
be = be.astype(float)

be = be.stack(level = 1)
be = be.reorder_levels([0, 2, 1], axis = 0).sort_index(axis = 0)

pv = pv.stack(level = 1)
pv = pv.reorder_levels([0, 2, 1], axis = 0).sort_index(axis = 0)

be.to_excel('output//' + mod + '_asset_age.xlsx')

display((df_stata['b'].loc['X'].applymap(dp4) * (df_stata['p'].loc['X'] >= 1)).T)
display_side_by_side(pv.loc['1X'])

year          8       9  10 11 12 13      14 15      16 17
sec smpl                                                  
1   0     0.0811  0.0689                         0.0386   
    9     0.1190                                 0.0505   
    10            0.1250                                  
    11    0.1120                      0.1500              
    12    0.0607  0.0948                         0.0450

sec 
 1 
 
 
 
 year 
 8 
 9 
 10 
 11 
 12 
 13 
 14 
 15 
 16 
 17 
 
 
 smpl 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 p 
 3 
 3 
 0 
 0 
 0 
 0 
 0 
 0 
 2 
 0 
 
 
 9 
 p 
 3 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 2 
 0 
 
 
 10 
 p 
 0 
 3 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 11 
 p 
 2 
 0 
 0 
 0 
 0 
 0 
 3 
 0 
 0 
 0 
 
 
 12 
 p 
 1 
 3 
 0 
 0 
 0 
 0 
 0 
 0 
 2 
 0

## <부표 2b>

In [27]:
mod = 'ms_3_0_0_ea_4'

df_stata = get_duchin(mod, sec = [1], smpl = fs, ys = range(8, 18))

be = df_stata.unstack().stack(level = 0)
pv = be.loc[idx[['X', 'id'], ['p']], idx[1, fs, range(8, 18)]]
be = be.loc[idx[['X', 'id'], ['b', 'e']], idx[1, fs, range(8, 18)]]
be = be.rename(index=order_dict, level = 0).sort_index(axis = 0)
pv = pv.rename(index=order_dict, level = 0).sort_index(axis = 0)
be = be.astype(float)

be = be.stack(level = 1)
be = be.reorder_levels([0, 2, 1], axis = 0).sort_index(axis = 0)

pv = pv.stack(level = 1)
pv = pv.reorder_levels([0, 2, 1], axis = 0).sort_index(axis = 0)

be.to_excel('output//' + mod + '_asset_age.xlsx')

display((df_stata['b'].loc['X'].applymap(dp4) * (df_stata['p'].loc['X'] >= 1)).T)
display_side_by_side(pv.loc['1X'])

year     8        9       10       11       12 13       14 15 16 17
sec smpl                                                           
1   0        -0.1190  0.1430  -0.1770                              
    9        -0.1620          -0.1780  -0.1320                     
    10                                                             
    11                0.1880                       -0.2330         
    12                        -0.1860

sec 
 1 
 
 
 
 year 
 8 
 9 
 10 
 11 
 12 
 13 
 14 
 15 
 16 
 17 
 
 
 smpl 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 p 
 0 
 1 
 2 
 3 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 9 
 p 
 0 
 2 
 0 
 2 
 1 
 0 
 0 
 0 
 0 
 0 
 
 
 10 
 p 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 11 
 p 
 0 
 0 
 2 
 0 
 0 
 0 
 2 
 0 
 0 
 0 
 
 
 12 
 p 
 0 
 0 
 0 
 2 
 0 
 0 
 0 
 0 
 0 
 0